# Simulation Boxes

The goal of `freud` is to perform generic analyses of particle simulations.
Such simulations are always conducted within some region representing physical space; in `freud`, these regions are known as *simulation boxes*, or simply *boxes*.
An important characteristic of many simulations is that the simulation box is periodic, *i.e.* particles can travel and interact across system boundaries (for more information, see [the Wikipedia page](https://en.wikipedia.org/wiki/Periodic_boundary_conditions)).
Simulations frequently use periodic boundary conditions to effectively simulate infinite systems without actually having to include an infinite number of particles.
In such systems, a box in N dimensions can be represented by N linearly independent vectors.

The `Box` class provides the standard API for such simulation boxes throughout `freud`.
The class represents some 2- or 3-dimensional region of space, and it provides utility functions for interacting with this space, including the ability to wrap vectors outside this box into the box according to periodic boundary conditions.
A 3d (2d) simulation box is fully defined by 3 (2) linearly independent vectors, which are represented within the Box class according to 3 (2) characteristic lengths as well as 3 (1) tilt factor indicating how these vectors are angled with respect to one another.
In this notebook, we demonstrate the basic features of the `Box` class, particularly the facility for wrapping particles back into the box under periodic boundary conditions.
For the interested reader, complete documentation of the Box class is available on the [official `freud` documentation page](https://freud.readthedocs.io/en/latest/box.html).

# Box basics

## Box Creation

There are many ways to construct a box.
We demonstrate all of these below, with some discussion of when they might be useful.

### Default (full) API

Boxes may be constructed explicitly using all arguments.
Such construction is useful when performing *ad hoc* analyses involving custom boxes.
In general, boxes are assumed to be 3D and [orthorhombic](https://en.wikipedia.org/wiki/Orthorhombic_crystal_system) unless otherwise specified.

In [3]:
from freud import box

# All of the below examples are valid boxes.
box = box.Box(Lx=5, Ly=6, Lz=7, xy=0.5, xz=0.6, yz=0.7, is2D=False)
box = box.Box(1, 3, 2, 0.3, 0.9)
box = box.Box(5, 6, 7)
box = box.Box(5, 6, is2D=True)
box = box.Box(5, 6, xy=0.5, is2D=True)

### Factory Functions

In order to simplify the creation of `freud.box.Box` objects from other representations of boxes, we provide a number of factory functions for conversion from these representations.

In [4]:
#box_data = some_object.box
#box_from_box = box.Box.from_box(box_data)
#box_matrix = to_matrix(some_object.box)
#box_from_matrix = box.Box.from_matrix(box_matrix)

### Convenience APIs

Many simulation boxes have some degree of symmetry, so we have provided the following convenience constructors for common box geometries.

In [3]:
#If your box is a cube, you may use the following:
cube_box = box.Box.cube(L=5)
print(cube_box.Lx, cube_box.Lz, cube_box.Ly, cube_box.xy, cube_box.xz, cube_box.yz, cube_box.is2D())

5.0 5.0 5.0 0.0 0.0 0.0 False


In [ ]:
#If your box is a square, you may use the following:
square_box = box.Box.square(L=5)
print(square_box.Lx, square_box.Lz, square_box.Ly, square_box.xy, square_box.xz, square_box.yz, square_box.is2D())

### From existing box

Some data readers may provide box objects which can be directly passed in:

# Export

If you want to export or display the box, you can use the following:

## `to_matrix()`

In [6]:
square_box = box.Box.square(L=5)
square_box.to_matrix()

[[5.0, 0.0, 0.0], [0, 5.0, 0.0], [0, 0, 0.0]]

## `to_tuple()`

You may also export as a named tuple (which can be used to initialize another `freud` box):

In [7]:
square_box.to_tuple()

BoxTuple(Lx=5.0, Ly=5.0, Lz=0.0, xy=0.0, xz=0.0, yz=0.0)